# Torrent Downloader

Paste your magnet link and run the cell.

In [ ]:
import sys
import subprocess
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'libtorrent==2.0.11'])

import libtorrent as lt
import time
import os

MAGNET_LINK = ''
SAVE_PATH = '/content/torrents'

if not MAGNET_LINK:
    print('⚠️ Set MAGNET_LINK variable above')
else:
    os.makedirs(SAVE_PATH, exist_ok=True)
    
    trackers = [
        'udp://tracker.opentrackr.org:1337/announce',
        'udp://tracker.torrent.eu.org:451/announce',
        'udp://exodus.desync.com:6969/announce',
        'udp://tracker.openbittorrent.com:6969/announce'
    ]
    magnet = MAGNET_LINK
    for tr in trackers:
        magnet += f'&tr={tr}'
    
    session = lt.session({'enable_dht': True, 'enable_lsd': True})
    params = lt.parse_magnet_uri(magnet)
    params.save_path = SAVE_PATH
    handle = session.add_torrent(params)
    
    print('📡 Fetching metadata...')
    while not handle.status().has_metadata:
        time.sleep(1)
    
    print(f'✅ {handle.status().name}')
    print(f'📦 Size: {handle.status().total_wanted/(1024**3):.2f} GB')
    print('⬇️ Downloading...')
    
    while not handle.status().is_seeding:
        s = handle.status()
        print(f'\r{s.progress*100:.1f}% | ↓{s.download_rate/1024:.0f} KB/s | Peers: {s.num_peers}', end='')
        time.sleep(2)
    
    print('\n🎉 Download complete!')
    print(f'📁 Files saved to: {SAVE_PATH}')